In [89]:
import numpy as np
import sympy as sp
import functions as fn
from IPython.display import display, Math
import scipy.integrate as int
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import math

hbar = 1
me   = 1
mp = 1836.15
u = (me*mp)/(me+mp)
u=1
Z = 1
alfa = 1/137
c = 1/alfa

w,r, phi, theta , phi_p,theta_p = sp.symbols('w r phi theta phi_p theta_p', real=True)

dtheta = sp.lambdify(theta,sp.sin(theta),"numpy")


hart2cm = 455.633 # en angstroms
hart2ev = 27.2116 # en electronvoltios
hart2invcm = 219475

"""
Importamos las funciones definidas del anterior taller para asi usar los autoestados del atomo de hidrogeno
"""


def eferic_armonics_numeric(var1,var2,l,m):
    ef = (sp.sqrt((2*l+1)/(4*sp.pi) * sp.factorial(l-abs(m))/sp.factorial(l+abs(m)) )
          * sp.exp(sp.I * m * var2) * 1/(2**l * sp.factorial(l)) *
          (1-w**2)**(abs(m)/2)* sp.diff((w**2-1)**l,(w,l+abs(m))))
    ef = ef.subs(w,sp.cos(var1))
    return ef

def Radial_part_numeric(var1,n,l):
    p = 2*Z*u/n  * var1
    radial = sp.S(0)
    for k in range(n-l):
        radial +=  (-1)**(k+1) * (sp.factorial(n+l))**2/ (sp.factorial(n-l-1-k)*sp.factorial(2*l+1+k)*sp.factorial(k)) * p**k

    radial *= - sp.sqrt(((2*Z*u/n)**3 * sp.factorial(n-l-1)/(2*n*(sp.factorial(n+l))**3  ))) * sp.exp(-p/2) * p**l

    return radial


def integrate_2d(Z, x, y, method):
    """
    Z: 2D array shaped (len(y), len(x))  -> f(y_i, x_j)
    x: 1D array (len Nx)
    y: 1D array (len Ny)
    method: 'rect', 'trapz', or 'simps'
    """
    if method == 'rect':
        dx = x[1]-x[0]
        dy = y[1]-y[0]
        return np.sum(Z) * dx * dy
    elif method == 'trapz':
        return np.trapezoid(np.trapezoid(Z, x=x, axis=1), x=y, axis=0)
    elif method == 'simps':
        return int.simpson(int.simpson(Z, x=x, axis=1), x=y, axis=0)
    else:
        raise ValueError("method must be 'rect','trapz' or 'simps'")


def safe_lambdify(vars, expr):
    f = sp.lambdify(vars, expr, "numpy")
    return lambda *args: np.broadcast_to(f(*args), np.broadcast(*args).shape)

def Psi(var1,var2,var3,n,l,m):
    psi = Radial_part_numeric(var1,n,l)*eferic_armonics_numeric(var2,var3,l,m)
    return psi


def L2_op (f):
    L2 = -hbar**2*(sp.diff(f,theta,2)   +  sp.cot(theta)*sp.diff(f,theta)+1/(sp.sin(theta))**2 *sp.diff(f,phi,2))
    return L2

def R_op(f):
    rop = -hbar**2/(2*u)* (sp.diff(f,r,2) + 2/r * sp.diff(f,r))
    return rop

def Hamiltonian(f):
    Ham = R_op(f) + 1/(2*u*r**2)*L2_op(f) -Z/r * f
    return Ham


def J2_op (f):
    j2 = L2_op(theta,phi,f) + L2_op(theta_p,phi_p,f) + 2*(Lx_op(theta,phi,f)+Ly_op(theta,phi,f)+Lz_op(theta,phi,f))* \
                                                         (Lx_op(theta_p,phi_p,f)+Ly_op(theta_p,phi_p,f)+Lz_op(theta_p,phi_p,f))
    return j2

def Jz_op (f):
    jz = Lz_op(theta,phi,f) + Lz_op(theta_p,phi_p,f)
    return jz


def clebseth_G(j1,j2,m1,m2,j,m):
    return abs(sp.integrate(sp.integrate( sp.conjugate(Y1_Y2(j1,m1,j2,m2)) * eferic_armonics_numeric(theta,phi,j,m) *sp.sin(theta),(theta,0,sp.pi)),(phi,0,2*sp.pi)))

def Y1_Y2  (l,m,lp,mp):
    mult_esferic = eferic_armonics_numeric(theta,phi,l,m) * eferic_armonics_numeric(theta_p,phi_p,lp,mp)
    return mult_esferic

def Lx_op (var1,var2,f):
    Lx = sp.I*hbar*(-sp.sin(var2)*sp.diff(f,var1) + sp.cot(var1)*sp.cos(var2)*sp.diff(f,var2))
    return Lx

def Ly_op (var1,var2,f):
    Ly = -sp.I*hbar*(sp.cos(var2)*sp.diff(f,var1)   -  sp.cot(var1)*sp.sin(var2)*sp.diff(f,var2))
    return Ly

def Lz_op (var1,var2,f):
    Lz = -sp.I*hbar*sp.diff(f,var2)
    return Lz

def Lplus_op(var1,var2,f):
    Lplus = Lx_op(var1,var2,f) + sp.I*Ly_op(var1,var2,f)
    return Lplus

def Lminus_op(var1,var2,f):
    Lminus = Lx_op(var1,var2,f) - sp.I*Ly_op(var1,var2,f)
    return Lminus


"""
para comprobar el teorema del virial (2<T> = <V>) para el caso del atomo de hidrogeno, calcularemos las componentes de matriz de la energia cinetica y potencial.
"""

#Comenzamos definiendo los elementos de matriz usando los autoestados del atomo de hidrogeno
#pero comenzando definiendo los operadores energia cinetica y potencial a partir del hamiltoniano.

def T(f):
    kine = Hamiltonian(f)- (-Z/r * f)
    return kine

def V(f):
    V = Hamiltonian(f)- T(f)
    return V


for j in range(1,10):
    display (Math(r"<\Psi"+f"_{{{j}{0}{0}}}|"+r"\hat{T}|\Psi"+f"_{{{j}{0}{0}}}> =" + sp.latex(round(sp.integrate(sp.integrate(sp.integrate(sp.conjugate(Psi(r,theta,phi,j,0,0))*2*T(Psi(r,theta,phi,j,0,0))*r**2*sp.sin(theta),(r,0,sp.oo)),(theta,0,sp.pi)),(phi,0,2*sp.pi))*hart2ev,2)   ) ))

print("\n")

for j in range(1,10):
    display (Math(r"<\Psi"+f"_{{{j}{0}{0}}}|"+r"\hat{V}|\Psi"+f"_{{{j}{0}{0}}}> =" + sp.latex(round(sp.integrate(sp.integrate(sp.integrate(sp.conjugate(Psi(r,theta,phi,j,0,0))*(-1)*V(Psi(r,theta,phi,j,0,0))*r**2*sp.sin(theta),(r,0,sp.oo)),(theta,0,sp.pi)),(phi,0,2*sp.pi))*hart2ev,2)   ) ))



<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>

In [90]:
# Para hallar las correcciones relativistas aplicamos el metodo de perturbaciones, el cual permite hallar energías de la perturbación a partir de las autofunciones del hamiltoniano sin perturbar.

#Para optimizar recursos, ya sabemos que la primera correccion mezcla autoestados de distinto n y de distinto l, por lo que recorreremos una matriz para cada n , una matriz de (n-1)X(n-1),


def T2(f):
    # return Hamiltonian(sp.simplify(Hamiltonian(f) + (Z/r * f))) + Z/r * (Hamiltonian(f) + (Z/r * f))
    return Hamiltonian(sp.sympify(Hamiltonian(f)+ Z/r *f))+sp.sympify(Z/r*(Hamiltonian(f)+ Z/r *f))

def En (n):
    return -u/2 * (Z**2/n**2)

num = 5


for n in range(1,num):
    T2matr = np.zeros((n,n))
    for l1 in range(n):
        for l2 in range(l1,n):

            # T2_num = safe_lambdify((theta,phi),sp.integrate(sp.conjugate(Psi(r,theta,phi,n,l1,0))*T2(Psi(r,theta,phi,n,l2,0)),(r,0,sp.oo)))


            T2matr[l1,l2] = round(-1/(2*c**2) * ((En(n))**2*sp.integrate(sp.integrate(sp.integrate(sp.conjugate(Psi(r,theta,phi,n,l1,0))*Psi(r,theta,phi,n,l2,0)*r**2*sp.sin(theta),(r,0,sp.oo)),(theta,0,sp.pi)),(phi,0,2*sp.pi)) +

            2*sp.integrate(sp.integrate(sp.integrate(sp.conjugate(Psi(r,theta,phi,n,l1,0))*Hamiltonian(Psi(r,theta,phi,n,l2,0))*r**2*sp.sin(theta),(r,0,sp.oo)),(theta,0,sp.pi)),(phi,0,2*sp.pi))*sp.integrate(sp.integrate(sp.integrate(sp.conjugate(Psi(r,theta,phi,n,l1,0))*Z*Psi(r,theta,phi,n,l2,0)*r*sp.sin(theta),(r,0,sp.oo)),(theta,0,sp.pi)),(phi,0,2*sp.pi)) +

             sp.integrate(sp.integrate(sp.integrate(sp.conjugate(Psi(r,theta,phi,n,l1,0))*Z*Psi(r,theta,phi,n,l2,0)*sp.sin(theta),(r,0,sp.oo)),(theta,0,sp.pi)),(phi,0,2*sp.pi)))* hart2invcm,7)

            # T2matr[l1,l2] = np.round(abs(integrate_2d(T2_num(Tvals,Pvals)*dtheta(Tvals),thetvals,phivals,'simps')) ,3) * hart2invcm
            # T2matr[l1,l2] = -1/(2*c**2) * ((En(n))**2 + 2*En(n)*(Z*u/n**2) + Z**2*u**2/(n**3*(l1+1/2)) ) * hart2invcm


    T2matr[1:num,0:num-1] = T2matr[0:num-1,1:num].T
    print(T2matr)

#Comparando los resultados que se obtienen son iguales al resultado analitico:


print("\n")

for n in range(1,num):
    T2matr = np.zeros((n,n))
    for l1 in range(n):
        T2matr[l1,l1] = -1/(2*c**2) * ((En(n))**2 + 2*En(n)*(Z*u/n**2) + Z**2*u**2/(n**3*(l1+1/2)) ) * hart2invcm

    T2matr[1:num,0:num-1] = T2matr[0:num-1,1:num].T
    print(T2matr)

[[-7.3084275]]
[[-1.1876195  0.       ]
 [ 0.        -0.2131625]]
[[-0.3789555  0.         0.       ]
 [ 0.        -0.0902275  0.       ]
 [ 0.         0.        -0.0324819]]
[[-0.1655816   0.          0.          0.        ]
 [ 0.         -0.0437744   0.          0.        ]
 [ 0.          0.         -0.019413    0.        ]
 [ 0.          0.          0.         -0.00897241]]


[[-7.30842746]]
[[-1.18761946  0.        ]
 [ 0.         -0.21316247]]
[[-0.3789555  0.         0.       ]
 [ 0.        -0.0902275  0.       ]
 [ 0.         0.        -0.0324819]]
[[-0.16558156  0.          0.          0.        ]
 [ 0.         -0.04377444  0.          0.        ]
 [ 0.          0.         -0.01941301  0.        ]
 [ 0.          0.          0.         -0.0089724 ]]


In [150]:
#Ahora haremos lo mismo para el estado n=3 y l = 1 para la correccion relativista spin orbita:

# Definimos la matriz spin orbita como :

def SPO(f,ms):
    if ms == 1/2:
        spo = 1/2 * alfa**2 * Z/r**3 * (Lz_op(theta,phi,f)*1/2 + 1/2*(Lplus_op(theta,phi,f)*hbar))
    elif ms == -1/2:
        spo = 1/2 * alfa**2 * Z/r**3 * (-Lz_op(theta,phi,f)*1/2*hbar + 1/2*(Lminus_op(theta,phi,f)*hbar))

    return spo

#Para n = 3 y l =1 tenemos una matriz 2x2 en la que se acoplan los estados para l-1/2 y l+1/2
n = 3
l = 1

LSmatrix = np.zeros((3,3))
thetvals,phivals = np.linspace(0.00001,np.pi-0.0001,500) , np.linspace(0.00001,2*np.pi-0.00001,500)
Tvals, Pvals = np.meshgrid(thetvals, phivals) # x, y

for m1 in range(-l,l+1):
    for m2 in range(m1,l+1):

        a = sp.sympify(sp.integrate(sp.conjugate(Psi(r,theta,phi,n,l,m1))* SPO(Psi(r,theta,phi,n,l,m2),1/2)*r**2,(r,0,sp.oo)))
        b = sp.lambdify((theta,phi),a,'numpy')

        LSmatrix[m1+l,m2+l] = abs(integrate_2d(b(Tvals,Pvals)*dtheta(Tvals),thetvals,phivals,'simps'))*hart2invcm
LSmatrix[1:3,0:3-1] = LSmatrix[0:3-1,1:3].T


eig1, eig2 = np.linalg.eig(LSmatrix)

print(eig1)







[-0.04180452  0.0778955   0.03609079]
